In [5]:
#library import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import calendar
from scipy.stats import iqr
import sys

In [2]:
#Read the file and return the dataframe
def read_file(filename):
    df1=pd.read_csv(filename,header=None)
    df1.columns=['DeviceID','poller_time','target_id','measured_value','metric_name']
    df1=df1.drop(['DeviceID'],axis=1)
    return df1
#Returns 25th percentile of an array/series
def q1(x):
    return x.quantile(0.25)
#Returns 75th percentile of an array/series
def q2(x):
    return x.quantile(0.75)
#Transform the timestamp and use aggregation functions to group metrics by hour and day of week, for every device-metric combination
def df_t(df):
    df['poller_time']=df['poller_time'].apply(lambda x:datetime.datetime.fromtimestamp(x))
    df['hour']=df['poller_time'].apply(lambda x: x.hour)
    df['dayofweek']=df['poller_time'].apply(lambda x: x.weekday())
    fdf=df.groupby(['target_id','metric_name','hour','dayofweek']).agg(['mean', 'min','max','median',q1,q2]).reset_index()
    return fdf

In [6]:
#call the functions
new_df=read_file('combined0917-1001.csv')
#Transform the dataframe with aggregations
trans_df=df_t(new_df)
#Assign Column names
trans_df.columns=['target_id','metric_name','hour','dayofweek','mean','min','max','median','q1','q2']

In [18]:
#Develop device and metric pairs
column_df=trans_df.groupby(['target_id','metric_name'])['max'].size().reset_index().rename({'max':'threshold'},axis=1)
#For every pair of device-metric find the threshold
for i in range(column_df.shape[0]):
    #Index alll values of device-metric combo
    max1=trans_df.loc[(trans_df['target_id']==column_df.loc[i,'target_id'])&(trans_df['metric_name']==column_df.loc[i,'metric_name']),'max']
    #Get IQR and top whisker
    quar_75=q2(max1)
    iqr1=iqr(max1)
    #Determine the threshold
    #Outlier=1.5*IQR
    if(column_df.loc[i,'target_id']=='utilization'):
        column_df.loc[i,'threshold']=max(95,quar_75+iqr1*1.5)
    else:
        column_df.loc[i,'threshold']=quar_75+iqr1*1.5

In [20]:
#Write to file
column_df.to_csv('threshold.csv')